In [60]:
from __future__ import division
import os, warnings, tensorflow as tf
from utils.model import * 
import numpy as np

In [61]:
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

## Before we begin

This notebook is a preliminary public code repository to the paper "Stateful Detection of Model Extraction Attacks".

Our code base borrows from the following other repositories:

* **StealML** of Tramer _et al._ [1] - [https://github.com/ftramer/Steal-ML/](https://github.com/ftramer/Steal-ML/)
* **ActiveThief** of Pal _et al._ [2] - [https://bitbucket.org/iiscseal/activethief/](https://bitbucket.org/iiscseal/activethief/)

The AdvPD attacks in Papernot _et al._ [3] are reimplemented by us.

The Fashion-MNIST dataset is courtesy Han Xiao _et al._ [4], made available on their [Github repository](https://github.com/zalandoresearch/fashion-mnist) under a MIT License.

### License

The MIT License (MIT) Copyright ©

1. **Fashion MNIST:** [2017] Zalando SE, https://tech.zalando.com

2. **ActiveThief:** 2019 Soham Pal, Yash Gupta, Aditya Shukla, Aditya Kanade, Shirish Shevade, Vinod Ganapathy. Indian Institute of Science.

    THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

## Load datasets

In [62]:
from dsl.fashionmnist_dsl import FashionMNISTDSL

### Partition classes

In [63]:
partition_seed = 1337

np.random.seed(partition_seed)

train_dsl = FashionMNISTDSL(batch_size=100, mode='test', normalize_channels=False, shuffle_each_epoch=False, seed=666)
chosen_classes = list(np.random.choice(np.arange(train_dsl.num_classes), train_dsl.num_classes//2, False))
remaining_classes = list(set(list(np.arange(train_dsl.num_classes))) - set(chosen_classes))

Loading test data
datasets/fashionmnist/t10k-images-idx3-ubyte
(10000, 28, 28, 1)
(10000,)


In [64]:
chosen_classes

[6, 3, 0, 4, 5]

In [65]:
remaining_classes

[8, 1, 2, 9, 7]

### Load confidential dataset $\mathcal{D}_C$

In [66]:
train_dsl = FashionMNISTDSL(batch_size = 100, mode='train', normalize_channels=False, shuffle_each_epoch=False, seed=666, keep_class=chosen_classes)
val_dsl   = FashionMNISTDSL(batch_size = 100, mode='val', normalize_channels=False, shuffle_each_epoch=False, seed=666, keep_class=chosen_classes)

Creating DSL only for classes 6 3 0 4 5
Loading train data
datasets/fashionmnist/train-images-idx3-ubyte
(60000, 28, 28, 1)
(60000,)
processing...


Creating DSL only for classes 6 3 0 4 5
Loading val data
datasets/fashionmnist/train-images-idx3-ubyte
(60000, 28, 28, 1)
(60000,)
processing...


### Load benign users

In [67]:
PD_test_dsl  = FashionMNISTDSL(batch_size = 100, mode='test', normalize_channels=False, shuffle_each_epoch=False, seed=666, keep_class=chosen_classes)

Creating DSL only for classes 6 3 0 4 5
Loading test data
datasets/fashionmnist/t10k-images-idx3-ubyte
(10000, 28, 28, 1)
(10000,)
processing...


In [68]:
AltPD_test_dsl  = FashionMNISTDSL(batch_size = 100, mode='test', normalize_channels=False, shuffle_each_epoch=False, seed=666, keep_class=remaining_classes)

Creating DSL only for classes 8 1 2 9 7
Loading test data
datasets/fashionmnist/t10k-images-idx3-ubyte
(10000, 28, 28, 1)
(10000,)
processing...


### Obtain dataset parameters

In [69]:
width, height, channels = train_dsl.get_sample_shape()

is_multilabel = train_dsl.is_multilabel()
num_classes = train_dsl.get_num_classes()
num_batches = train_dsl.get_num_batches()

### Load attacker datasets $\mathcal{D}_A$

Syn-Uniform Retraining (Tramer _et al._ [1])

In [70]:
from dsl.uniform_dsl import UniformDSL

sample_shape = (width, height, channels)
uniform_dsl = UniformDSL(batch_size=100, mode='train', shape=sample_shape, sample_limit=1000, seed=666)

ImageNet samples

In [71]:
from dsl.imagenet_dsl import ImagenetDSL

resize = (width, height)
img_noise_train_dsl = ImagenetDSL(batch_size=100, mode='train', resize=resize, normalize_channels=True, num_to_keep=1000, start_batch=1, end_batch=1, seed=666)

Loading train data
datasets/Imagenet64/train_data_batch_1.npy


## Create models

In [72]:
import tensorflow as tf
from models.deepcnn import DeepCNN
from models.vae import NewHSVNVAE

In [73]:
from cfg import cfg, config

In [74]:
with tf.variable_scope("vae", reuse=tf.AUTO_REUSE):
    vae = NewHSVNVAE(
            batch_size=cfg.batch_size,
            height=height, width=width,
            channels=channels,
            num_classes=2,
            is_training=False,
            z_size=32,
            random_draws=1,
            noise_mean=5.0
         )

Tensor("vae_2/Decoder/conv2d_transpose/LeakyRelu:0", shape=(?, 2, 2, 256), dtype=float32)
Tensor("vae_2/Decoder/conv2d_transpose_1/LeakyRelu:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("vae_2/Decoder/conv2d_transpose_2/LeakyRelu:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("vae_2/Decoder/conv2d_transpose_3/LeakyRelu:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("vae_2/Decoder/conv2d_transpose_4/Sigmoid:0", shape=(?, 28, 28, 1), dtype=float32)


In [75]:
saver = tf.train.Saver()

In [76]:
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

In [77]:
saver.restore(sess, 'modeldir/fashionmnist-1337/model_epoch_100')

INFO:tensorflow:Restoring parameters from modeldir/fashionmnist-1337/model_epoch_100


INFO:tensorflow:Restoring parameters from modeldir/fashionmnist-1337/model_epoch_100


## Outlier Dataset Generation

Here, we demonstrate the outlier dataset $\mathcal{D}_O$ generation procedure. Samples are shown in the next section.

In [78]:
import copy

In [79]:
outlier_dsl = copy.deepcopy(train_dsl)
num_train_samples = outlier_dsl.data.shape[0]

locs = np.random.uniform(0.0, 1.0, size=num_train_samples)
noise_generated = np.concatenate([[np.random.normal(loc=loc, scale=0.5, size=outlier_dsl.data.shape[1:]) for loc in locs]])
vnoise   = np.random.uniform(low=0.2, high=0.3, size=num_train_samples )
vnoise   = vnoise.reshape(-1, 1, 1, 1)

outlier_dsl.data = np.clip(vnoise * noise_generated + (1-vnoise) * outlier_dsl.data, 0.0, 1.0)

## Load AdvPD samples

In [80]:
advpd_dsl = copy.deepcopy(train_dsl)
advpd_dsl.data = np.load('datasets/AdvPD.npy')

## Dataset Previews

Before we show the representation in latent space, let us take a look at the original dataset samples.

In [81]:
from matplotlib import pyplot as plt
%matplotlib inline

### Confidential dataset $\mathcal{D}_C$ samples

In [82]:
import os
import matplotlib.pyplot as plt
import numpy as np

# Create a folder named 'train_dsl' if it doesn't exist
folder_name = 'train_dsl'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

for i in range(1, 100 * 10 + 1):
    fig = plt.figure(figsize=(1, 1))  # Create a new figure for each image
    img = np.random.randint(10, size=(10, 10))
    plt.imshow(train_dsl.data[i - 1].squeeze(-1), cmap='gray')
    plt.axis('off')
    plt.savefig(os.path.join(folder_name, 'image_{}.png'.format(i)), bbox_inches='tight', pad_inches=0)  # Save each image in the 'train_dsl' folder
    plt.close()  # Close the current figure to release memory


In [89]:
import os
import matplotlib.pyplot as plt
import numpy as np

# Create a folder named 'train_dsl' if it doesn't exist
folder_name = 'val_dsl'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

for i in range(1, 100 * 10 + 1):
    fig = plt.figure(figsize=(1, 1))  # Create a new figure for each image
    img = np.random.randint(10, size=(10, 10))
    plt.imshow(val_dsl.data[i - 1].squeeze(-1), cmap='gray')
    plt.axis('off')
    plt.savefig(os.path.join(folder_name, 'image_{}.png'.format(i)), bbox_inches='tight', pad_inches=0)  # Save each image in the 'train_dsl' folder
    plt.close()  # Close the current figure to release memory


## PD Benign User samples

In [83]:
import os
import matplotlib.pyplot as plt
import numpy as np

# Create a folder named 'PD_test_dsl' if it doesn't exist
folder_name = 'PD_test_dsl'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

for i in range(1, 100 * 10 + 1):
    fig = plt.figure(figsize=(1, 1))  # Create a new figure for each image
    img = np.random.randint(10, size=(10, 10))
    plt.imshow(PD_test_dsl.data[i - 1].squeeze(-1), cmap='gray')
    plt.axis('off')
    plt.savefig(os.path.join(folder_name, 'image_{}.png'.format(i)), bbox_inches='tight', pad_inches=0)  # Save each image in the 'PD_test_dsl' folder
    plt.close()  # Close the current figure to release memory


## AltPD Benign User samples

In [84]:
import os
import matplotlib.pyplot as plt
import numpy as np

# Create a folder named 'AltPD_test_dsl' if it doesn't exist
folder_name = 'AltPD_test_dsl'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

for i in range(1, 100 * 10 + 1):
    fig = plt.figure(figsize=(1, 1))  # Create a new figure for each image
    img = np.random.randint(10, size=(10, 10))
    plt.imshow(AltPD_test_dsl.data[i - 1].squeeze(-1), cmap='gray')
    plt.axis('off')
    plt.savefig(os.path.join(folder_name, 'image_{}.png'.format(i)), bbox_inches='tight', pad_inches=0)  # Save each image in the 'AltPD_test_dsl' folder
    plt.close()  # Close the current figure to release memory


### Outlier dataset $\mathcal{D}_O$ samples

In [85]:
import os
import matplotlib.pyplot as plt
import numpy as np

# Create a folder named 'outlier_dsl' if it doesn't exist
folder_name = 'outlier_dsl'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

for i in range(1, 100 * 10 + 1):
    fig = plt.figure(figsize=(1, 1))  # Create a new figure for each image
    img = np.random.randint(10, size=(10, 10))
    plt.imshow(outlier_dsl.data[i - 1].squeeze(-1), cmap='gray')
    plt.axis('off')
    plt.savefig(os.path.join(folder_name, 'image_{}.png'.format(i)), bbox_inches='tight', pad_inches=0)  # Save each image in the 'outlier_dsl' folder
    plt.close()  # Close the current figure to release memory


### Syn-Uniform Retraining attacker dataset $\mathcal{D}_A$ samples

In [86]:
import os
import matplotlib.pyplot as plt
import numpy as np

# Create a folder named 'uniform_dsl' if it doesn't exist
folder_name = 'uniform_dsl'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

for i in range(1, 100 * 10 + 1):
    fig = plt.figure(figsize=(1, 1))  # Create a new figure for each image
    img = np.random.randint(10, size=(10, 10))
    plt.imshow(uniform_dsl.data[i - 1].squeeze(-1), cmap='gray')
    plt.axis('off')
    plt.savefig(os.path.join(folder_name, 'image_{}.png'.format(i)), bbox_inches='tight', pad_inches=0)  # Save each image in the 'uniform_dsl' folder
    plt.close()  # Close the current figure to release memory


### AdvPD-JSMA attacker dataset $\mathcal{D}_A$ samples

In [87]:
import os
import matplotlib.pyplot as plt
import numpy as np

# Create a folder named 'advpd_dsl' if it doesn't exist
folder_name = 'advpd_dsl'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

for i in range(1, 100 * 10 + 1):
    fig = plt.figure(figsize=(1, 1))  # Create a new figure for each image
    img = np.random.randint(10, size=(10, 10))
    plt.imshow(advpd_dsl.data[150:][i - 1].squeeze(-1), cmap='gray')
    plt.axis('off')
    plt.savefig(os.path.join(folder_name, 'image_{}.png'.format(i)), bbox_inches='tight', pad_inches=0)  # Save each image in the 'advpd_dsl' folder
    plt.close()  # Close the current figure to release memory


### NPD-ActiveThief attacker dataset $\mathcal{D}_A$ samples

In [88]:
import os
import matplotlib.pyplot as plt
import numpy as np

# Create a folder named 'img_noise_train_dsl' if it doesn't exist
folder_name = 'img_noise_train_dsl'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

for i in range(1, 100 * 10 + 1):
    fig = plt.figure(figsize=(1, 1))  # Create a new figure for each image
    img = np.random.randint(10, size=(10, 10))
    plt.imshow(img_noise_train_dsl.data[i - 1].squeeze(-1), cmap='gray')
    plt.axis('off')
    plt.savefig(os.path.join(folder_name, 'image_{}.png'.format(i)), bbox_inches='tight', pad_inches=0)  # Save each image in the 'img_noise_train_dsl' folder
    plt.close()  # Close the current figure to release memory


## VAE Visualizations

Finally, let us take a look at the latent representations for various datasets.

In [ ]:
print(vae)

In [ ]:
print(train_dsl)

In [ ]:
print(sess)

In [ ]:
true_train_zs = sample_z(vae, train_dsl, sess)
outlier_zs    = sample_z(vae, outlier_dsl, sess)
uniform_zs    = sample_z(vae, uniform_dsl, sess)
advpd_zs      = sample_z(vae, advpd_dsl, sess)
img_noise_zs  = sample_z(vae, img_noise_train_dsl, sess)

In [ ]:
pd_zs = sample_z(vae, PD_test_dsl, sess)
altpd_zs = sample_z(vae, AltPD_test_dsl, sess)

In [ ]:
labels  = [ 'Confidential', 'Outlier' ]
plot_pca([true_train_zs, outlier_zs], labels=labels, num_samples=1000, markers=['o', 'x'])

In [ ]:
labels  = [ 'Confidential', 'Syn' ]
plot_pca([true_train_zs, uniform_zs], labels=labels, num_samples=1000, markers=['o', 'x'])

In [ ]:
labels  = [ 'Confidential', 'AdvPD seed', 'AdvPD' ]
plot_pca([true_train_zs, advpd_zs[:150], advpd_zs[150:]], labels=labels, num_samples=1000, markers=['o', 'x', 'x'], facecolors=[False, True, True])

In [ ]:
labels  = ['Confidential', 'NPD']
plot_pca([true_train_zs, img_noise_zs], labels=labels, num_samples=1000, markers=['o', 'x'])

In [ ]:
labels  = [ 'Confidential', 'Outlier', 'AltPD' ]
plot_pca([true_train_zs, outlier_zs, altpd_zs], labels=labels, num_samples=1000, markers=['o', 'x', 'x'], facecolors=[False, True, True])

In [ ]:
labels  = [ 'Confidential', 'Outlier', 'PD' ]
plot_pca([true_train_zs, outlier_zs, pd_zs], labels=labels, num_samples=1000, markers=['o', 'x', 'x'], facecolors=[False, True, True])

## References

[1] Tramer, F.; Zhang, F.; Juels, A.; Reiter, M. K.; and Ristenpart, T. 2016.  
Stealing Machine Learning Models via Prediction APIs.  
In _USENIX Security 16_.

[2] Pal, S.; Gupta, Y.; Shukla, A.; Kanade, A.; Shevade, S. K.; and Ganapathy, V. 2020.  
ActiveThief: Model Extraction Using Active Learning and Unannotated Public Data.  
In _Proceedings of the Thirty-Fourth AAAI Conference on Artificial Intelligence, AAAI’20. AAAI Press_.

[3] Papernot, N.; McDaniel, P.; Goodfellow, I.; Jha, S.; Celik, Z. B.; and Swami, A. 2017.  
Practical Black-Box Attacks against Machine Learning.  
In _Proceedings of the 2017 ACM on Asia Conference on Computer and Communications Security, ASIA CCS '17_. ACM.

[4] Xiao, H.; Rasul, K.; and Vollgraf, R. 2017.  
Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning Algorithms.  
_CoRR_ abs/1708.07747. URL http://arxiv.org/abs/1708.07747.